In [ ]:
#import 
import tensorflow as tf
import os
import numpy as np
import inspect

#list for vgg_mean, https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3-keras implementation
VGG_MEAN = [103.939, 116.779, 123.68]

class vggnet:
    def __init__(self, vgg16_npy_path=None):
        #download weights,link = 'https://www.cs.toronto.edu/~frossard/vgg16/vgg16_weights.npz'
        if vgg16_npy_path=None:
            path = inspect.getfile(vggnet)
            path = os.path.abspath(os.path.join(path, os.pardir))
            path = os.path.join(path, "vgg16_weights.npz")
            vgg16_npy_path = path
            
        self.data_dict = np.load(vgg16_npy_path,encoding='latin1').item()
        print(path)
        
        
         def max_pool(self,bottom,name):
        return tf.nn.max_pool(bottom,ksize=[1,2,2,1],stride =[1,2,2,1],padding= 'SAME',name = name)
    
    def conv_layer(self,bottom,name):
        with tf.variable_scope(name):
            #read https://www.tensorflow.org/programmers_guide/variable_scope,weights and biases in downloaded file
            filters = self.get_conv_filter(name)
            conv = tf.nn.conv2d(bottom,filter,[1,1,1,1],padding='SAME')
            conv_biases = self.get_bias(name)
            conv = tf.nn.bias_add(conv,conv_biases)
            relu = tf.nn.relu(conv)
            return relu
        
    def fc_layer(self,bottom,name):
        with tf.variable_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
            dim* = d
            x = tf.reshape(bottom,[-1,dim])
            
            weights = self.get_fc_weight(name)
            bias = self.get_bias(name)
            fc = tf.nn.bias_add(tf.matmul(x,weights),bias)
            
            return fc
        
    def get_conv_filter(self,name):
        return tf.constants(self.data_dict[name][0],name = 'filter')
    
    def get_bias(self,name):
        return tf.constants(self.data_dict[name][1],name = 'biases')
    
    def get_fc_weight(self,name):
        return tf.constants(self.data_dict[name][0],name = 'weights')
        
            

    def build(self,rgb):
        start_time = time.time()
        print("model build started")
        rgb_scaled = rgb*255.0
        
        red,green,blue = tf.split(axis =3,num_or_size_splits = 3,value = rgbscaled)
        #Why use get_shape().as_list()? read http://stackoverflow.com/questions/40666316/how-to-get-tensorflow-tensor-dimensions-shape-as-int-values
        assert red.get_shape().as_list()[1:] = [224,224,1] 
        assert green.get_shape().as_list()[1:] = [224,224,1] 
        assert blue.get_shape().as_list()[1:] = [224,224,1] 
        
        bgr = tf.concat(values=[blue - VGG_MEAN[0],green - VGG_MEAN[1],red - VGG_MEAN[2]],axis=3)
        
        #tf.shape(bgr)
        
        assert bgr.get_shape().as_list()[1:] == [224, 224, 3]
        
        self.conv1_1 = self.conv_layer(bgr,"conv1_1")
        self.conv1_2 = self.conv_layer(self.conv1_1,"conv1_2")
        self.pool1 = self.max_pool(self.conv1_2,"pool1")
        
        self.conv2_1 = self.conv_layer(self.pool1,"conv2_1")
        self.conv2_2 = self.conv_layer(self.conv2_1,"conv2_2")
        self.pool2 = self.max_pool(self.conv2_2,"pool2")
        
        self.conv3_1 = self.conv_layer(self.pool2,"conv3_1")
        self.conv3_2 = self.conv_layer(self.conv3_1,"conv3_2")
        self.conv3_3 = self.conv_layer(self.conv3_2,"conv3_3")
        self.pool3 = self.max_pool(self.conv3_3,"pool3")
        
        self.conv4_1 = self.conv_layer(self.pool3,"conv4_1")
        self.conv4_2 = self.conv_layer(self.conv4_1,"conv4_2")
        self.conv4_3 = self.conv_layer(self.conv4_2,"conv4_3")
        self.pool4 = self.max_pool(self.conv4_3,"pool4")
        
        self.conv5_1 = self.conv_layer(self.pool4,"conv5_1")
        self.conv5_2 = self.conv_layer(self.conv4_1,"conv5_2")
        self.conv3_3 = self.conv_layer(self.conv4_2,"conv5_3")
        self.pool5 = self.max_pool(self.conv4_3,"pool5")
        
        self.fc6 = self.fc_layer(self.pool5,"fc6")
        self.relu6 = tf.nn.relu(self.fc6)
        
        self.fc7 = self.fc_layer(self.relu6,"fc7")
        self.relu7 = tf.nn.relu(self.fc7)
        
        self.fc8 = self.fc_layer(self.relu7,"fc8")
        self.prob = tf.nn.softmax(self.fc8,name='prob')
        
        print("build complete")
        
        
   
        
    
            
            